In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pickle
import csv
import glob

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger
import tensorflow as tf

# Data Loading

In [2]:
Sampled_train = pd.read_csv("dataset/train.csv")
Sampled_test = pd.read_csv("dataset/test.csv")
Sampled_cv = pd.read_csv('dataset/cv.csv')

# Data Preprocessing

In [3]:
# Drop some mysterious fault type
Sampled_train.drop(Sampled_train[(Sampled_train.faultNumber == 3) | (Sampled_train.faultNumber == 9) | (Sampled_train.faultNumber == 15)].index, inplace = True)
Sampled_test.drop(Sampled_test[(Sampled_test.faultNumber == 3) | (Sampled_test.faultNumber == 9) | (Sampled_test.faultNumber == 15)].index, inplace = True)
Sampled_cv.drop(Sampled_cv[(Sampled_cv.faultNumber == 3) | (Sampled_cv.faultNumber == 9) | (Sampled_cv.faultNumber == 15)].index, inplace = True)

# make the Y value usable in LSTM
y_train = to_categorical(Sampled_train['faultNumber'],num_classes=21)
y_test = to_categorical(Sampled_test['faultNumber'],num_classes=21)
y_cv = to_categorical(Sampled_cv['faultNumber'],num_classes=21)

# drop unused meta data from x
x_train_df = Sampled_train.drop(['faultNumber','simulationRun','sample'],axis=1)
x_test_df = Sampled_test.drop(['faultNumber','simulationRun','sample'],axis =1)
x_cv_df = Sampled_cv.drop(['faultNumber','simulationRun','sample'],axis =1)

# Parameter

In [4]:
cost_false_positive = 2
cost_false_negative = 16

# ['xmv_10', 'xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmv_5', 'xmeas_17', 'xmeas_18', 'xmeas_9']

sensors = ['xmeas_19', 'xmv_9', 'xmv_4']
initial_model_path = 'models/evsi/' + '7_' + "xmv_11/"
evsi_counter = 7

# get prior probability
temp = Sampled_train['faultNumber'].value_counts()
non_fault = temp[0]
total = temp.sum()

temp = Sampled_cv['faultNumber'].value_counts()
non_fault += temp[0]
total += temp.sum()

P_present = non_fault/total
P_absent = 1 - P_present

# ground truth
ground_truth = Sampled_test['faultNumber'].tolist()

# Utility Functions

In [5]:
def feature_remover(features_names):
    # remove a list of features from x
    
    dimension = dict()
    
    # row dimension
    dimension['train_row'] = len(x_train_df)
    dimension['test_row'] = len(x_test_df)
    dimension['cv_row'] = len(x_cv_df)
    
    # create a copy so we don't change the original dataframe
    x_train_masked_df = x_train_df.copy()
    x_test_masked_df = x_test_df.copy()
    x_cv_masked_df = x_cv_df.copy()
    
    for feature in features_names:
        x_train_masked_df.drop([feature], axis = 1, inplace = True)
        x_test_masked_df.drop([feature], axis = 1, inplace = True)
        x_cv_masked_df.drop([feature], axis = 1, inplace = True)
        
    # column dimension
    dimension['train_col'] = x_train_masked_df.shape[1]
    dimension['test_col'] = x_test_masked_df.shape[1]
    dimension['cv_col'] = x_cv_masked_df.shape[1]
    
    standard_scalar = StandardScaler()
    x_train_masked_df = standard_scalar.fit_transform(x_train_masked_df)
    x_test_masked_df = standard_scalar.transform(x_test_masked_df)
    x_cv_masked_df = standard_scalar.transform(x_cv_masked_df)
    
    x_train = np.resize(x_train_masked_df, (dimension['train_row'], dimension['train_col'], 1))
    x_test = np.resize(x_test_masked_df, (dimension['test_row'], dimension['test_col'], 1))
    x_cv = np.resize(x_cv_masked_df, (dimension['cv_row'], dimension['cv_col'], 1))
    
    return dimension, x_train, x_test, x_cv

In [6]:
def train_model(x_train, y_train, x_cv, y_cv, train_col, feature_name, model_path):
    model = Sequential()
    model.add(LSTM(256,input_shape= (train_col, 1),return_sequences= True))
    model.add(LSTM(128,return_sequences= False))
    model.add(Dense(300))
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Dense(21,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print(model_path + feature_name)
    print(train_col)
    
    # training
    model.fit(x_train, y_train, epochs=35,verbose=0,batch_size=256,validation_data = (x_cv, y_cv))
    
    # saving the model
    model.save(model_path + feature_name)
    
    # saving the history
    model_paras = model.history
    with open(model_path + feature_name + '/history.pickle', 'wb') as handle:
        pickle.dump(model_paras.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return model

In [7]:
# helper function to calculate probability of correctly giving signal when present
def get_signal_present(prediction, ground_truth):
    present_index = list()
    for i in range(len(ground_truth)):
        if ground_truth[i] == 0:
            present_index.append(i)
    
    counter = 0
    for index in present_index:
        if prediction[index] == 0:
            counter += 1
    
    return counter/len(present_index)

# helper function to calculate probability of correctly giving signal when present
# there should be a more generic way using operator module to merge this with the one above.
def get_no_signal_absent(prediction, ground_truth):
    absent_index = list()
    for i in range(len(ground_truth)):
        if ground_truth[i] != 0:
            absent_index.append(i)
    
    counter = 0
    for index in absent_index:
        if prediction[index] != 0:
            counter += 1
    return counter/len(absent_index)

In [8]:
def get_expected_cost(prediction, ground_truth):
  # get P(signal|present) and P(no signal|absent)
    P_signal_present = get_signal_present(prediction, ground_truth)
    P_no_signal_absent = get_no_signal_absent(prediction, ground_truth)
    P_signal_absent = 1 - P_no_signal_absent
    P_no_signal_present = 1 - P_signal_present

  # get P(signal)
    P_signal = P_present * P_signal_present + P_absent * P_signal_absent
    P_no_signal = 1 - P_signal

  # bayesian probability
    P_absent_signal = (P_signal_absent * P_absent) / P_signal
    P_present_signal = (P_signal_present * P_present) / P_signal
    P_absent_no_signal = (P_no_signal_absent * P_absent) / P_no_signal
    P_present_no_signal = (P_no_signal_present * P_present) / P_no_signal

  #calculate the expected cost
    cost = P_signal * min(cost_false_positive * P_absent_signal, cost_false_negative * P_present_signal) + P_no_signal * min(cost_false_positive * P_absent_no_signal, cost_false_negative * P_present_no_signal)
  
    return cost

In [9]:
def plot_dict(dictionary):
    x, y = [], []
    for key, value in dictionary.items():
        x.append(key)
        y.append(value)
    return x, y

# Training, Calculating EVSI

In [10]:
model_path = initial_model_path
unused_sensor = list()

#write header
# with open('log/sensor_selection.csv', 'a', newline='') as out:
#     csv_out = csv.writer(out)
#     csv_out.writerow(['sensor', 'evsi'])

while len(sensors) > 1:
    unused_sensor.append(sensors.copy())
    #print(unused_sensor[evsi_counter])
    ################### lower branch ##############################################
    dimension, x_train, x_test, x_cv = feature_remover(features_names = sensors)
    
    base_model = train_model(x_train, y_train, x_cv, y_cv, dimension['train_col'], 'base', model_path)
    
    base_prediction = base_model.predict_classes(x_test, verbose = 0)
    
    base_cost = get_expected_cost(base_prediction, ground_truth)
    
    ################### upper branch ##############################################
    upper_dict = dict()
    evsi_dict = dict()
    
    for sensor in sensors:
        # create a list of sensor that needs to get removed
        remove_sensors = sensors.copy()
        remove_sensors.remove(sensor)
        
        # prepare x 
        dimension, x_train, x_test, x_cv = feature_remover(features_names = remove_sensors)
        
        # train the model associated with current sensor, save the model, get the prediction
        upper_model = train_model(x_train, y_train, x_cv, y_cv, dimension['train_col'], '+' + sensor, model_path)
        upper_prediction = upper_model.predict_classes(x_test, verbose = 0)
        
        # calculate evsi
        upper_dict[sensor] = get_expected_cost(upper_prediction, ground_truth)
        evsi_dict[sensor] = base_cost - upper_dict[sensor]
    
    ################### rank and others ##############################################
    # rank by evsi
    ranked_sensors = sorted(evsi_dict.items(), key=lambda x: x[1], reverse= True)
    sensor_to_monitor = ranked_sensors[0][0]
    
    # remove the top sensor from the list
    sensors.remove(sensor_to_monitor)
    
    # updating some parameters
    evsi_counter += 1
    model_path = 'models/evsi/' + str(evsi_counter) + '_' + sensor_to_monitor + '/'
    
    # logging the evsi over the current sensors
    with open('log/sensor_selection.csv', 'a', newline='') as out:
        csv_out = csv.writer(out)
        for row in ranked_sensors:
            csv_out.writerow(row)
        csv_out.writerow([])

models/evsi/7_xmv_11/base
49
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/evsi/7_xmv_11/base\assets
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
models/evsi/7_xmv_11/+xmeas_19
50
INFO:tensorflow:Assets written to: models/evsi/7_xmv_11/+xmeas_19\assets
models/evsi/7_xmv_11/+xmv_9
50
INFO:tensorflow:Assets written to: models/evsi/7_xmv_11/+xmv_9\assets
models/evsi/7_xmv_11/+xmv_4
50
INFO:tensorflow:Assets written to: models/evsi/7_xmv_11/+xmv_4\assets
models/evsi/8_xmeas_19/base
50
INFO:tensorflow:Assets written to: models/evsi/8_xmeas_19/base\assets
models/evsi/8_xmeas_19/+xmv_9
51
INFO:tensorflow:Assets written to: models/evs

# Forward Stepwise

In [11]:
FS_sensors = ['xmv_10', 'xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmv_5', 'xmeas_17', 'xmeas_18', 'xmeas_9']

In [12]:
# delete this for the next run
unused_sensor = list()
unused_sensor.append(['xmv_10', 'xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmv_5', 'xmeas_17', 'xmeas_18', 'xmeas_9'])
unused_sensor.append(['xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmv_5', 'xmeas_17', 'xmeas_18', 'xmeas_9'])
unused_sensor.append(['xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmv_5', 'xmeas_17', 'xmeas_18'])
unused_sensor.append(['xmv_11', 'xmeas_19', 'xmeas_21', 'xmv_9', 'xmv_4', 'xmeas_17', 'xmeas_18'])
unused_sensor.append(['xmv_11', 'xmeas_19', 'xmv_9', 'xmv_4', 'xmeas_17', 'xmeas_18'])
unused_sensor.append(['xmv_11', 'xmeas_19', 'xmv_9', 'xmv_4', 'xmeas_18'])
unused_sensor.append(['xmv_11', 'xmeas_19', 'xmv_9', 'xmv_4'])
unused_sensor.append(['xmeas_19', 'xmv_9', 'xmv_4'])
unused_sensor.append(['xmv_9', 'xmv_4'])

In [13]:
FS_counter = 0
sensor_to_monitor = 'none'

#write header
with open('log/acc_improvement.csv', 'a', newline='') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['sensor', 'acc_improvement'])

with open('log/acc.csv', 'a', newline='') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(['sensor', 'acc'])
    
while len(FS_sensors) > 1:
    ################### base accuracy ##############################################
    dimension, x_train, x_test, x_cv = feature_remover(features_names = FS_sensors)
    model_path = glob.glob('models/evsi/' + str(FS_counter) + '_*')[0] + '/'
    
    # evsi and fs match, just load the same model
    if set(FS_sensors) == set(unused_sensor[FS_counter]):
        base_model = load_model(model_path + 'base', compile = True)
        print(sensor_to_monitor + ': load')
        
    # evsi and fs doesn't match, train the model
    else:
        model_path = 'models/ml/' + str(FS_counter) + '_' + sensor_to_monitor + '/'
        print(sensor_to_monitor + ': train')
        base_model = train_model(x_train, y_train, x_cv, y_cv, dimension['train_col'], 'base', model_path)
        
        
    _, base_acc = base_model.evaluate(x_test, y_test, verbose = 0)
    
    ################### improved accuracy ##############################################
    acc_dict = dict()
    acc_improvement_dict = dict()
    for sensor in FS_sensors:

        # create a list of sensor that needs to get removed
        remove_sensors = FS_sensors.copy()
        remove_sensors.remove(sensor)

        # prepare x 
        dimension, x_train, x_test, x_cv = feature_remover(features_names = remove_sensors)
        
        # evsi and fs match, just load the same model
        if set(FS_sensors) == set(unused_sensor[FS_counter]):
            improved_model = load_model(model_path + '+' + sensor, compile = True)
        
        # evsi and fs doesn't match, train the model
        else:
            improved_model = train_model(x_train, y_train, x_cv, y_cv, dimension['train_col'], '+' + sensor, model_path)
            
        _, acc_dict[sensor] = improved_model.evaluate(x_test, y_test, verbose = 0)
        
        
        #calculate improvement
        acc_improvement_dict[sensor] = acc_dict[sensor] - base_acc
        
    ################### rank and others ##############################################
    
    #rank by accuracy improvement
    FS_ranked_sensors_by_improv = sorted(acc_improvement_dict.items(), key=lambda x: x[1], reverse= True)
    sensor_to_monitor = FS_ranked_sensors_by_improv[0][0]

    # remove the top sensor from the list
    FS_sensors.remove(sensor_to_monitor)

    # updating some parameters
    FS_counter += 1
    model_path = 'models/ml/' + str(FS_counter) + '_' + sensor_to_monitor + '/'

    # logging the improvement over the current sensors
    with open('log/acc_improvement.csv', 'a', newline='') as out:
        csv_out = csv.writer(out)
        for row in FS_ranked_sensors_by_improv:
            csv_out.writerow(row)
        csv_out.writerow([])
    
    
    # logging the accuracy over the current sensors
    FS_ranked_sensors_by_acc = sorted(acc_dict.items(), key=lambda x: x[1], reverse= True)
    with open('log/acc.csv', 'a', newline='') as out:
        csv_out = csv.writer(out)
        for row in FS_ranked_sensors_by_acc:
            csv_out.writerow(row)
        csv_out.writerow([])

none: load
xmv_10: load
xmeas_21: train
models/ml/2_xmeas_21/base
44
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/base\assets
models/ml/2_xmeas_21/+xmv_11
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmv_11\assets
models/ml/2_xmeas_21/+xmeas_19
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmeas_19\assets
models/ml/2_xmeas_21/+xmv_9
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmv_9\assets
models/ml/2_xmeas_21/+xmv_4
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmv_4\assets
models/ml/2_xmeas_21/+xmv_5
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmv_5\assets
models/ml/2_xmeas_21/+xmeas_17
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmeas_17\assets
models/ml/2_xmeas_21/+xmeas_18
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmeas_18\assets
models/ml/2_xmeas_21/+xmeas_9
45
INFO:tensorflow:Assets written to: models/ml/2_xmeas_21/+xmeas_9\assets
xmv_4: train
models/ml/3_xmv_4/